In [2]:
import numpy as np
from profiles import *
from dispersion import SphericalJeansDispersion
from likelihood import GaussianLikelihood
from fitter import MinuitFitter
from utils import *

In [3]:
homedir = '/home/andrea/Desktop/work/DWARF/Jvalue/project1/test/Isotrop_Core_nonPlum'
MockSize = 100
dataSize = 1000
dset = 1
# enter model choice - cf. casedir (options 1,2,3,4)
mod = 1
theta = 0.5
D = 100

In [4]:
data = load_gaia(homedir, MockSize, dataSize, dset, mod, D, True)

Using  /home/andrea/Desktop/work/DWARF/Jvalue/project1/test/Isotrop_Core_nonPlum/data/gs100_bs050_rcrs100_rarcinf_core_0400mpc3_df_1000_1.dat


In [5]:
kinematic = data[0:3]
rh = data[-3]
r0_true = data[-2]
rho0_true = data[-1]

In [6]:
dm = build_profile('nfw')
st = build_profile('plummer',**{'c':1,'rh':rh})
kr = build_kernel('iso')

In [7]:
dwarf_props = {'D':D,'theta':theta,'rt':np.inf,'errs':False}

In [8]:
Sigma = SphericalJeansDispersion(dm, st, kr,dwarf_props)

In [9]:
Sigma.params

{'J': 18,
 'dm_a': 1,
 'dm_b': 3,
 'dm_c': 1,
 'dm_r0': 1,
 'dm_rho0': 1,
 'st_c': 1,
 'st_rh': 1.0,
 'st_rhoh': 1}

In [10]:
LL = GaussianLikelihood(kinematic, Sigma)

In [11]:
LL.free_pars

{'J': 18}

In [12]:
#global global_loglike
global_loglike = LL
M = MinuitFitter(LL)

In [13]:
M.loglike.free_pars

{'J': 18}

In [13]:
M.settings

{'J': 18, 'errordef': 0.5, 'pedantic': False, 'print_level': 0}

In [15]:
#M.set_error('dm_r0',0.01)
M.set_error('J',0.01)
#M.set_bound('dm_r0',(kinematic[0].min(),kinematic[0].max()))
M.set_bound('J',(17,20))

ValueError: J not a parameter of logLike function.
The parameters are ['st_rh', 'dm_r0', 'st_c', 'dm_a', 'dm_b', 'dm_c', 'dm_rho0', 'st_rhoh']

In [15]:
M.settings

{'J': 18,
 'dm_r0': 1,
 'error_J': 0.01,
 'error_dm_r0': 0.01,
 'errordef': 0.5,
 'limit_J': (17, 20),
 'limit_dm_r0': (0.077103885381996154, 5.4674701279476601),
 'pedantic': False,
 'print_level': 0}

In [19]:
M.fit()

<function fitter.<lambda>>